# Reformatting the Excel: Assumptions and stuff



* Copied only vital information for the data generation.
* Everything is in its own cell, so easy to import into a pandas dataframe.
* Study number five (Jorge, 2015) had measurements for 6- and 12-week checkpoints for Pain, Function and QoL. Only 12 was added to the new excel, since assumedly that is the duration of the treatment/therapy.
* O'Reilly paper was removed since they reported only the change in QoL, not the actual baseline value. Can't calculate the percentages based on just that.
* Fransen paper has reported the WOMAC scores as inverse, so 0 represents the worst status and 100 the best. Might be that others have done that too
    * _The WOMAC scores were reverse scored (100 = no pain
or difficulty, 0 = extreme pain or difficulty), so that for all outcome measures higher scores are better scores._ https://www.jrheum.org/content/jrheum/28/1/156.full.pdf (That has been adjusted to the project_excel)


* Group:
    * Used 1 - exercise and 0 - control because it is also using in the objective functions

* No of sessions:
    * Also added because it is using in the objective functions
    * 0 used for control assuming control fact do not have any sessions

* Age:
    * several different "value types" in paranthesis
    * for some there is the +/- notation: not sure what that means (another notation for SD it turns out)
    * for those cells with input like 71.9 (69.3, 74.6), in the paranthesis seems to be the 95% CI (only checked Cheung paper)
    * for some there is mean (SD) notation used, SD being standard deviation
    * According to the Wallis paper age structure may be Age_Mean_value (Age_SD)
    * According to the Evcik paper the +/- notation follows SD

* BMI:
    * same issues as with age


* Study participants/subjects:
    * for simplicity's sake, it was assumed that all participants finished the therapies/participating in the study.
        * (In reality this is not true, but cutting corners here a little bit, this is already getting out of hand)
    * Do we need to keep Male and Female separate?  - _I don't think so. The information is not used in any of the objectives or in the data later, all the numbers are for the genders combined. Ideally, we should take the gender into account, but I think it is just complicating matters at this point. -J_

* Pain levels:
    * only the womac score included
    * unless otherwise mentioned, it was assumed that the score was 0-20
    * notice: not all were in the same scale, so needed to scale
    * WOMAC (35 pain) don't know what this means and can't get access to the article to check so not doing anything to the values --> is foin
    * 'changes' was interpreted to mean how much the original womac value changed, so the 'after' value would baseline+change.


* Functionality
    * can't get access to check what womac fc 85 means so leaving that as is (since googling didn't help either) --> again, is foin

    
* QoL
    * different tests were used to test quality of life. If the scales were same, it was assumemd that they are comparable as is.
    * everything was scaled to 0-100
    * assuming that NHP emotional reaction is scaled from 0-100
    * should maybe check if all scores are the "same order", like 0 is worst in all and 100 best. (Because for example for NHP: "the total score for each domain is 100 where a score of 0 indicates good subjective health status and 100 indicates poor subjective health status." and I think SF-36 might use the scale the other way, 0 being worst and 100 being best)
        * assuming 100 is the best score and 0 the worst
        * I inversed the last study's QoL scores since they were the wrong way around (0 being the best and 100 worst)
        * assuming others are OK, since the numbers make sense (they are rising after intervention/with time)
    * Also for one paper there is no values, what should be done with that? (marked in grey)
        * _This study has been dropped._


    * **In the final report we shouold mention that several different metrics were used and that they might not be comparable**
        * Although in theory, since we are comparing the change (or the percentage of improvement), they might not be that different?


Color coding for the half finished table:
* pink: need to somehow turn values into standard deviation
    * *KNN imputation was used, right?*
* green: need to find out what mm means and turn the value into scores (need to read the source paper)
    * still don't know what mm means, but those were just another scale



_TODO: add table columns and explain them_

#Data Preprocessing

Handling missing values using KNN imputation


*I commented it for now, because I don't have the file in my gdrive, maybe you could update it so that it will work here too? -J*

In [94]:
# import numpy as np
# import pandas as pd
# from sklearn.impute import KNNImputer

# #Access files from Google Drive
# from google.colab import drive
# drive.mount('/content/gdrive')

# df = pd.read_excel("/content/gdrive/MyDrive/TIES5830/project_excel.xlsx", header=0)

# print("Dataset:")
# print(df.head())

In [95]:
# # Display the number of missing values in each column
# print("Missing Values in Each Column:")
# print(df.isnull().sum())

In [96]:
# # Identify numeric columns
# numeric_columns = df.select_dtypes(include=[float, int]).columns
# print("Numeric Columns:", numeric_columns)

In [97]:
# # Create a DataFrame with only numeric columns for KNN imputation
# numeric_df = df[numeric_columns]

# # Initialize KNN Imputer
# imputer = KNNImputer(n_neighbors=3)  # Choose a suitable number of neighbors

# # Apply KNN imputation to the numeric columns
# imputed_numeric_data = imputer.fit_transform(numeric_df)

# # Convert back to a DataFrame with the original numeric column names
# imputed_numeric_df = pd.DataFrame(imputed_numeric_data, columns=numeric_columns)

In [98]:
# # Reattach non-numeric columns to the imputed data
# imputed_df = pd.concat([df[['reference']], imputed_numeric_df], axis=1)

# print("Imputed Data:")
# print(imputed_df.head)

# Data generation


* Find a method that can generate artificial data based on mean and variance
* If a method cannot be found, need to assume distriubtion
    * Compare that to some existing data (if there is such)
        * if looks good, run with it
        * but what if it does not look good? And if there is existing data, wouldn't it be easier to generate synthetic data based on that? There's ways to generate "anonymous" (synthetic) data from existing patient information, so that no patient confidiality is broken.

## Need to generate data for


* Age
    * assumption: distribution probably skewed to the older side, since it is knee OA (older people more likely to have it)
    * modeled as normal distribution, should maybe skew it a bit to the older side?
* BMI
    * Knee osteoarthiritis and bmi connected, could use that information?  https://www.sciencedirect.com/science/article/abs/pii/S1297319X11001370
    * modeled as a normal distribution, should skew it somehow maybe?
* Pain (baseline)
    * modeled as normal distribution for now (same for all the rest - should probably think more)
* Physical Functionality (baseline)
* QoL (baseline)
* Pain (after intervention)
* Physical Functionality (after intervention)
* QoL (after intervention)

In [99]:
# Importing the data
import pandas as pd

# Importing the excel into a dataframe
df = pd.read_excel('project_excel.xlsx', index_col='reference')

## Data generation: normally distributed




All generated data is stored in a dictionary, where the first column in excel (or the df index row) is the key and its value is another dataframe of the form


{ <br>
        'age': data_age, <br>
        'bmi': data_bmi, <br>
        'patients_num': num_of_participants,<br>
        'pain_before': pain_before_data,<br>
        'pain_after': pain_after_data,<br>
        'functionality_before': func_before_data,<br>
        'functionality_after': func_after_data,<br>
        'QoL_before': qol_before_data,<br>
        'QoL_after': qol_after_data<br>
}

### <span style="color:red">NOTE:</span>
<span style="color:red"> This is a very naive approach and does not work well at all. </span> BUT it is enough to start working on the surrogates. We need to be cleverer if we want usable results. -J





**Reminder:**
 * pain on scale 0-20 (0 meaning no pain and being the best value)
    * so pain_after should be assumed to be smaller than pain_before and calculating the percentage as (before-after)/before * 100
 * function 0-68 (0 being the best)
    * percentage calculated as above
 * QoL on scale 0-100, the bigger the better.
    * value is assumed to get higher after intervention/with time, so calculating the percentage as (after-before)/before * 100

*Note: If the percentage values are negative, it means the improvement has been negative.*

In [100]:
import numpy as np

# Function that takes the whole column as input, divides it into pairs with the relevant mean and std
# except the number of participants in the study is just an integer.
# Then generates the data based on the above distributions and returns it in a dictionary
def generate_data(info):
    age = (info['age_mean'], info['age_std'])
    bmi = (info['bmi_mean'], info['bmi_std'])
    num_of_participants = int(info['num_of_subjects'])
    pain_before = (info['pain_before_mean'], info['pain_before_std'])
    pain_after = (info['pain_after_mean'], info['pain_after_std'])
    func_before = (info['func_before_mean'], info['func_before_std'])
    func_after = (info['func_after_mean'], info['func_after_std'])
    qol_before = (info['QoL_before_mean'], info['QoL_before_std'])
    qol_after = (info['QoL_after_mean'], info['QoL_after_std'])

    # Generating the data
    data_age = np.random.normal(age[0], age[1], num_of_participants)
    data_bmi = np.random.normal(bmi[0], bmi[1], num_of_participants)
    pain_before_data = np.random.normal(pain_before[0], pain_before[1], num_of_participants)
    pain_after_data = np.random.normal(pain_after[0], pain_after[1], num_of_participants)
    func_before_data = np.random.normal(func_before[0], func_before[1], num_of_participants)
    func_after_data = np.random.normal(func_after[0], func_after[1], num_of_participants)
    qol_before_data = np.random.normal(qol_before[0], qol_before[1], num_of_participants)
    qol_after_data = np.random.normal(qol_after[0], qol_after[1], num_of_participants)

    # Calculating the improvement percent for each patient (pain, function and QoL)
    pain_improvement = (pain_before_data - pain_after_data)/pain_before_data * 100
    func_improvement = (func_before_data - func_after_data)/func_before_data * 100
    qol_improvement = (qol_after_data - qol_before_data)/qol_before_data * 100

    return {
        'age': data_age,
        'bmi': data_bmi,
        'patients_num': num_of_participants,
        'pain_before': pain_before_data,
        'pain_after': pain_after_data,
        'pain_improvement': pain_improvement,
        'func_before': func_before_data,
        'func_after': func_after_data,
        'func_improvement': func_improvement,
        'QoL_before': qol_before_data,
        'QoL_after': qol_after_data,
        'QoL_improvement': qol_improvement
    }


In [101]:
# A function for creating csv files for each exercise and control group
import csv
import os

def write_to_csv(data, filename, folder):
    os.makedirs(folder, exist_ok=True)
    filepath = os.path.join(folder, filename)

    with open(filepath, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['age', 'bmi', 'pain_before', 'pain_after', 'pain_improvement',
                         'func_before', 'func_after', 'func_improvement', 'QoL_before', 'QoL_after', 'QoL_improvement'])

        for i in range(data['patients_num']):
            writer.writerow([data['age'][i], data['bmi'][i],
                             data['pain_before'][i], data['pain_after'][i], data['pain_improvement'][i],
                             data['func_before'][i], data['func_after'][i], data['func_improvement'][i],
                             data['QoL_before'][i], data['QoL_after'][i], data['QoL_improvement'][i]])


In [102]:
# This piece of code generates the data. Now commented because it obviously overwrites the csv files each run
# and we do not wish to do that each time.
# TODO: At some point, need to make better data because this one will not work well. (See PROBLEMS below)

# This is the folder where the data is stuffed. The folder doesn't have to exist,
# if it does not exist, the folder will be created
folder = 'data_initial'

synthetic_data_dict = {} # Storing all generated data here
# for index, row in df.iterrows():
    # Generating data for each row in the df and adding that to the dict with the row index being the key
    # synthetic_data_dict[index] = generate_data(row)
    # filename = f"data_{index}.csv"
    # write_to_csv(synthetic_data_dict[index], filename, folder)

# <span style="color:red">PROBLEMS</span>
* Negative values in the objective function data, that should not be allowed to happen since the values aren't realistic.
* Improvement rates are bloody ridiculous, they are not realistic at all. I think it is because of the negative values in the data, they are messing with the improvement calculations. Need to weed out the negative values and perhaps introduce some sort of correlation between the before and after levels.

### CSV shenanigans

In [103]:
# Here importing the data from csv files into dataframes

# the folder where the data is hanging out
# this will import ALL the .csv files within that folder
directory = "./data_initial"

dataframes_dict = {}

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename) # path to the file
        df = pd.read_csv(filepath)

        # Key is the filename without the .csv ending
        dataframes_dict[filename[:-4]] = df

In [104]:
dataframes_dict['data_an_2008_control'].head()

,age,bmi,pain_before,pain_after,pain_improvement,func_before,func_after,func_improvement,QoL_before,QoL_after,QoL_improvement
0,66.147821,31.770124,6.932760,8.520719,-22.905143,11.747159,18.905453,-60.936382,67.071871,69.083039,2.998526
1,63.843777,21.802928,5.173638,12.572855,-143.017659,6.246227,30.754903,-392.375708,71.991398,85.024808,18.104122
2,69.458339,24.669889,7.547620,8.450204,-11.958537,9.040084,34.294863,-279.364405,77.029963,56.682978,-26.414377
3,72.827657,20.508080,8.210735,-2.539558,130.929722,23.622446,31.354798,-32.733068,49.017837,68.534876,39.816199
4,62.370409,22.346341,7.745284,1.179480,84.771633,28.632475,2.003867,93.001418,66.795140,67.676838,1.320002


## Data generation: GAN

*Again, commented this the GAN stuff, since we don't wanna be creating new files each time we run the notebook. Also, the gdrive thing don't work on my end.. :D -J*

In [105]:
# #!pip install numpy pandas tensorflow

# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Dense
# from tensorflow.keras.models import Model
# import os

# #Access files from Google Drive
# from google.colab import drive
# drive.mount('/content/gdrive')

# df = pd.read_excel("/content/gdrive/MyDrive/project_excel.xlsx", header=0)

In [106]:
# # Define GAN components
# def build_generator(latent_dim, output_dim):
#     model_input = Input(shape=(latent_dim,))
#     x = Dense(64, activation='relu')(model_input)
#     x = Dense(128, activation='relu')(x)
#     x = Dense(output_dim, activation='linear')(x)
#     model = Model(model_input, x)
#     return model

# def build_discriminator(input_dim):
#     model_input = Input(shape=(input_dim,))
#     x = Dense(128, activation='relu')(model_input)
#     x = Dense(64, activation='relu')(x)
#     x = Dense(1, activation='sigmoid')(x)
#     model = Model(model_input, x)
#     return model

# def build_gan(generator, discriminator):
#     discriminator.compile(optimizer='adam', loss='binary_crossentropy')
#     discriminator.trainable = False
#     gan_input = Input(shape=(latent_dim,))
#     generated_data = generator(gan_input)
#     gan_output = discriminator(generated_data)
#     gan = Model(gan_input, gan_output)
#     gan.compile(optimizer='adam', loss='binary_crossentropy')
#     return gan

In [107]:
# # Training function
# def train_gan(generator, discriminator, gan, data, epochs=50, batch_size=16):
#     for epoch in range(epochs):
#         # Train discriminator
#         noise = np.random.normal(0, 1, (batch_size, latent_dim))
#         generated_data = generator.predict(noise)

#         real_data = data[np.random.randint(0, data.shape[0], batch_size)]
#         combined_data = np.vstack((real_data, generated_data))
#         labels = np.array([1] * batch_size + [0] * batch_size)

#         d_loss = discriminator.train_on_batch(combined_data, labels)

#         # Train generator
#         noise = np.random.normal(0, 1, (batch_size, latent_dim))
#         misleading_labels = np.array([1] * batch_size)

#         g_loss = gan.train_on_batch(noise, misleading_labels)

#         if epoch % 100 == 0:
#             print(f'Epoch {epoch} - Discriminator Loss: {d_loss}, Generator Loss: {g_loss}')

# # Define parameters
# latent_dim = 10
# output_dim = 10

# # Create directories for saving datasets
# os.makedirs("synthetic_datasets", exist_ok=True)

# # Iterate over each reference to generate synthetic datasets
# for _, row in df.iterrows():
#     ref = row['reference']
#     num_subjects = row['num_of_subjects']

#     real_data = np.array([
#         np.full(num_subjects, row['group']),
#         np.full(num_subjects, row['no_of_sessions']),
#         np.random.normal(row['age_mean'], row['age_std'], num_subjects),
#         np.random.normal(row['bmi_mean'], row['bmi_std'], num_subjects),
#         np.random.normal(row['pain_before_mean'], row['pain_before_std'], num_subjects),
#         np.random.normal(row['pain_after_mean'], row['pain_after_std'], num_subjects),
#         np.random.normal(row['func_before_mean'], row['func_before_std'], num_subjects),
#         np.random.normal(row['func_after_mean'], row['func_after_std'], num_subjects),
#         np.random.normal(row['QoL_before_mean'], row['QoL_before_std'], num_subjects),
#         np.random.normal(row['QoL_after_mean'], row['QoL_after_std'], num_subjects)
#     ]).T

#     generator = build_generator(latent_dim, output_dim)
#     discriminator = build_discriminator(output_dim)
#     gan = build_gan(generator, discriminator)

#     train_gan(generator, discriminator, gan, real_data, epochs=1000, batch_size=16)

#     noise = np.random.normal(0, 1, (num_subjects, latent_dim))
#     synthetic_data = generator.predict(noise)

#     synthetic_df = pd.DataFrame(synthetic_data, columns=[
#         'group', 'no_of_sessions', 'age', 'bmi', 'pain_before', 'pain_after', 'func_before', 'func_after', 'QoL_before', 'QoL_after'
#     ])

#     synthetic_df['group'] = synthetic_df['group'].round().astype(int)
#     synthetic_df['no_of_sessions'] = synthetic_df['no_of_sessions'].round().astype(int)

#     synthetic_df.to_csv(f"synthetic_datasets/{ref}_synthetic.csv", index=False)
#     print(f"Generated synthetic dataset for {ref}")

# print("All synthetic datasets have been generated.")

# Data generation with covariances

## <span style="color:red">NOTE</span>
<span style="color:red">This will be left half done, since it was done in parallel in case GAN did not render good results. It did tho, so no need to finish this!</span> 





Since the random data is crap and GAN might not pan out, let's just assume the correlation and go with that.

So, cutting some corners here, just assuming the correlations. Should randomly generate them somehow, so that the surrogates fit differently and also to make the results more realistic (since in real life they would not correlate similarly.) Perhaps the correlation could be calculated from the info we already have, would have to make some assumptions tho. Which is better, maybe it don't even matter for the results?

Code is mostly copy-pasted from above and here: https://scipy-cookbook.readthedocs.io/items/CorrelatedRandomSamples.html

### TODO
- make the correlations plausible. Now they are just silly chatGPT generated values.
    - age and BMI might correlate a little, older means higher BMI
    - age and BMI both might cause the baseline values to be higher (positive correlation)
        - except with the QoL, since it is inversed and most likely imporvement in pain and function will lead to better QoL so it might correlate with that
    - age and BMI might cause the pain and functionality after to get better results (i.e. lower bmi and age will give better results, so more dropping in the value, getting closer to 0.)



In [108]:
import numpy as np

# Function that takes the whole column as input, divides it into pairs with the relevant mean and std
# except the number of participants in the study is just an integer.
# Then generates the correlated data using the correlation matrix generated anew for each generation round
def generate_data_with_correlation(info):
    age = (info['age_mean'], info['age_std'])
    bmi = (info['bmi_mean'], info['bmi_std'])
    num_of_participants = int(info['num_of_subjects'])
    pain_before = (info['pain_before_mean'], info['pain_before_std'])
    pain_after = (info['pain_after_mean'], info['pain_after_std'])
    func_before = (info['func_before_mean'], info['func_before_std'])
    func_after = (info['func_after_mean'], info['func_after_std'])
    qol_before = (info['QoL_before_mean'], info['QoL_before_std'])
    qol_after = (info['QoL_after_mean'], info['QoL_after_std'])


    num_samples = num_of_participants

    # The mean values of samples
    mu = np.array([age[0], bmi[0], pain_before[0], func_before[0], qol_before[0], pain_after[0], func_after[0], qol_after[0]])

    # The covariance matrix. Right now the values are randomly picked. Should be cleverer and also maybe make different
    # correlation matrixes for each dataset/study?
    r = np.array([
    [1.0, 0.1, 0.1, 0.15, 0.15, -0.2, -0.3, -0.3],    # Variance of age
    [0.1, 1.0, 0.1, 0.15, 0.15, -0.2, -0.3, -0.3],    # Variance of BMI
    [0.1, 0.1, 1.0, 0.4, 0.4, -0.3, -0.4, -0.4],      # Covariance between age and pain_before, func_before, qol_before
    [0.15, 0.15, 0.4, 1.0, 0.7, -0.5, -0.6, -0.7],    # Covariance between BMI and pain_before, func_before, qol_before
    [0.15, 0.15, 0.4, 0.7, 1.0, -0.5, -0.6, -0.7],    # Variance of pain_before, func_before, qol_before
    [-0.2, -0.2, -0.3, -0.5, -0.5, 1.0, 0.8, 0.7],    # Covariance between age and pain_after, func_after, qol_after
    [-0.3, -0.3, -0.4, -0.6, -0.6, 0.8, 1.0, 0.9],    # Covariance between BMI and pain_after, func_after, qol_after
    [-0.3, -0.3, -0.4, -0.7, -0.7, 0.7, 0.9, 1.0]     # Variance of pain_after, func_after, qol_after
    ])



    # Generate the random samples.
    rng = np.random.default_rng()
    y = rng.multivariate_normal(mu, r, size=num_samples)

    print(y)

    # Generating the data
    data_age = np.random.normal(age[0], age[1], num_of_participants)
    data_bmi = np.random.normal(bmi[0], bmi[1], num_of_participants)
    pain_before_data = np.random.normal(pain_before[0], pain_before[1], num_of_participants)
    pain_after_data = np.random.normal(pain_after[0], pain_after[1], num_of_participants)
    func_before_data = np.random.normal(func_before[0], func_before[1], num_of_participants)
    func_after_data = np.random.normal(func_after[0], func_after[1], num_of_participants)
    qol_before_data = np.random.normal(qol_before[0], qol_before[1], num_of_participants)
    qol_after_data = np.random.normal(qol_after[0], qol_after[1], num_of_participants)

    # Calculating the improvement percent for each patient (pain, function and QoL)
    pain_improvement = (pain_before_data - pain_after_data)/pain_before_data * 100
    func_improvement = (func_before_data - func_after_data)/func_before_data * 100
    qol_improvement = (qol_after_data - qol_before_data)/qol_before_data * 100

    return {
        'age': data_age,
        'bmi': data_bmi,
        'patients_num': num_of_participants,
        'pain_before': pain_before_data,
        'pain_after': pain_after_data,
        'pain_improvement': pain_improvement,
        'func_before': func_before_data,
        'func_after': func_after_data,
        'func_improvement': func_improvement,
        'QoL_before': qol_before_data,
        'QoL_after': qol_after_data,
        'QoL_improvement': qol_improvement
    }


In [109]:
# Importing the excel into a dataframe
df = pd.read_excel('project_excel.xlsx', index_col='reference')

In [110]:
# This piece of code generates the data.

# This is the folder where the data is stuffed. The folder doesn't have to exist,
# if it does not exist, the folder will be created
folder = 'data_correlated'

correlated_synthetic_data_dict = {} # Storing all generated data here
# for index, row in df.iterrows():
#     # Generating data for each row in the df and adding that to the dict with the row index being the key
#     correlated_synthetic_data_dict[index] = generate_data(row)
#     filename = f"data_{index}.csv"
#     write_to_csv(correlated_synthetic_data_dict[index], filename, folder)

In [111]:
dataframes_dict['data_an_2008_control'].head()

,age,bmi,pain_before,pain_after,pain_improvement,func_before,func_after,func_improvement,QoL_before,QoL_after,QoL_improvement
0,66.147821,31.770124,6.932760,8.520719,-22.905143,11.747159,18.905453,-60.936382,67.071871,69.083039,2.998526
1,63.843777,21.802928,5.173638,12.572855,-143.017659,6.246227,30.754903,-392.375708,71.991398,85.024808,18.104122
2,69.458339,24.669889,7.547620,8.450204,-11.958537,9.040084,34.294863,-279.364405,77.029963,56.682978,-26.414377
3,72.827657,20.508080,8.210735,-2.539558,130.929722,23.622446,31.354798,-32.733068,49.017837,68.534876,39.816199
4,62.370409,22.346341,7.745284,1.179480,84.771633,28.632475,2.003867,93.001418,66.795140,67.676838,1.320002


# GAN-generated data from CSV to dictionary
Generation was originally done in another code file. Code added above 

In [112]:
# Here importing the data from csv files into dataframes

# the folder where the data is hanging out
# this will import ALL the .csv files within that folder
directory = "./GAN_Generated"

dataframes_dict_gan = {}

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename) # path to the file
        df = pd.read_csv(filepath)

        # Calculating the improvement percent for each patient (pain, function and QoL)
        df['pain_improvement'] = (df['pain_before'] - df['pain_after'])/df['pain_before'] * 100
        df['func_improvement'] = (df['func_before'] - df['func_after'])/df['func_after'] * 100
        df['QoL_improvement'] = (df['QoL_after'] - df['QoL_before'])/ df['QoL_before'] * 100

        # Key is the filename without the .csv ending
        dataframes_dict_gan[filename[:-4]] = df
        

In [113]:
dataframes_dict_gan['an_2008_exercise_synthetic'].head()

,group,no_of_sessions,age,bmi,pain_before,pain_after,func_before,func_after,QoL_before,QoL_after,pain_improvement,func_improvement,QoL_improvement
0,5,44,77.117580,14.539472,5.588644,5.520327,21.988897,22.629564,70.022480,73.527466,1.222432,-2.831106,5.005515
1,6,43,74.211500,13.635116,5.348004,5.092411,21.597746,21.567600,67.242170,71.313020,4.779228,0.139774,6.054013
2,4,37,63.971367,11.288850,4.698689,4.055223,18.586061,18.242400,57.515636,61.197197,13.694597,1.883858,6.400974
3,4,36,62.499893,12.081700,4.425854,4.508512,17.868702,18.509598,56.733430,59.383293,-1.867606,-3.462506,4.670726
4,6,51,88.805970,16.424040,6.568262,5.840533,25.406647,25.530130,80.040184,84.478540,11.079476,-0.483676,5.545160


# Surrogates



I'm not quite sure how we are supposed to fit the surrogates. I think it would be best to just fit it to the exercise groups, since that the performance of that is what we are interested in. I don't really know how we even could utilize the control groups here.

So, I will only be fitting the surrogates to the exercise group datasets here. Also, I am assuming that the point is to see if age and bmi affect the results (meaning the improvement percentage)

Also not sure if the performance metric R^2 was a good choice, it was just convenient

*Should probably choose at least 2, maybe even 3 or 4 and do some analysis on those so we can find out which works the best.*
- Chosen surrogates for further evaluation and testing:
    - SVM
    - Random Forest
    - Gradient Boost Regression
    

## Extracting the data and scaling it

* Fitting the surrogates only for the exercise groups.
* Scaling the data since some of the methods (like SVM) work better with scaled data (sensitivity to outliers etc)

### Normally distrubuted data (i.e. the crappy data)

In [114]:
# Extracting only the dataframes about the exercise groups

# These are the keys for the exercise group dataframse
ex_keys = [k for k in dataframes_dict.keys() if "exercise" in k]
exercise_dataframes_dict = {}
for key in ex_keys:
    exercise_dataframes_dict[key] = dataframes_dict[key].copy()


In [115]:
print("We have", len(exercise_dataframes_dict.keys()), "different therapies in the data")
print("data_an_2008_exercise: first few rows of the df below. It's not good...")
exercise_dataframes_dict['data_an_2008_exercise'].head()

We have 11 different therapies in the data
data_an_2008_exercise: first few rows of the df below. It's not good...


,age,bmi,pain_before,pain_after,pain_improvement,func_before,func_after,func_improvement,QoL_before,QoL_after,QoL_improvement
0,65.095656,23.163583,7.950896,2.963848,62.723098,6.277190,11.304656,-80.091022,72.351706,77.230457,6.743103
1,72.284960,28.579256,7.610465,-5.586427,173.404544,0.974135,11.278822,-1057.829350,56.919931,83.668522,46.993364
2,49.170737,22.842774,10.570031,5.500836,47.958188,21.831200,0.997755,95.429682,84.001932,79.968487,-4.801610
3,63.127090,27.460322,8.936780,8.036707,10.071564,19.192646,33.250728,-73.247228,55.577144,89.931924,61.814584
4,68.207988,26.477881,8.732150,8.677388,0.627123,3.472480,-5.232589,250.687392,71.616582,71.130541,-0.678672


In [116]:
from sklearn.preprocessing import StandardScaler

# First should scale all the data. I'm assuming that all the data is somewhat similar so just one scaler for all is enough, don't need to
# create one for each dataframe. Also need to remember to unscale the data in the end and only one scaler makes it easier
scaler = StandardScaler()

for key in exercise_dataframes_dict.keys():
    arr = scaler.fit_transform(exercise_dataframes_dict[key])
    exercise_dataframes_dict[key] = pd.DataFrame(arr, columns=exercise_dataframes_dict[key].columns)

exercise_dataframes_dict['data_an_2008_exercise'].head()

,age,bmi,pain_before,pain_after,pain_improvement,func_before,func_after,func_improvement,QoL_before,QoL_after,QoL_improvement
0,-0.151850,-0.795178,0.379069,-0.357032,0.063516,-0.552321,0.051199,-0.085742,0.100589,0.077832,-0.091386
1,0.597498,1.267427,0.295995,-2.338790,1.338126,-0.902961,0.049655,-1.644422,-1.562384,0.600112,1.522323
2,-1.811718,-0.917360,1.018204,0.230984,-0.106517,0.476116,-0.564806,0.194068,1.356051,0.299951,-0.554235
3,-0.357035,0.841272,0.619650,0.818741,-0.542820,0.301654,1.362834,-0.074832,-1.707087,1.108224,2.116533
4,0.172551,0.467101,0.569715,0.967237,-0.651583,-0.737770,-0.937171,0.441575,0.021370,-0.417017,-0.388939


Now the dataset has a mean of 0 and standard deviation of 1. Not sure if this is the best way to scale the data

### GAN-generated

In [117]:
# Extracting only the dataframes about the exercise groups

# These are the keys for the exercise group dataframse
ex_keys_gan = [k for k in dataframes_dict_gan.keys() if "exercise" in k]
exercise_dataframes_dict_gan = {}
for key in ex_keys_gan:
    exercise_dataframes_dict_gan[key] = dataframes_dict_gan[key].copy()

print("We have", len(exercise_dataframes_dict_gan.keys()), "different therapies in the data")
exercise_dataframes_dict_gan['an_2008_exercise_synthetic'].head()

We have 11 different therapies in the data


,group,no_of_sessions,age,bmi,pain_before,pain_after,func_before,func_after,QoL_before,QoL_after,pain_improvement,func_improvement,QoL_improvement
0,5,44,77.117580,14.539472,5.588644,5.520327,21.988897,22.629564,70.022480,73.527466,1.222432,-2.831106,5.005515
1,6,43,74.211500,13.635116,5.348004,5.092411,21.597746,21.567600,67.242170,71.313020,4.779228,0.139774,6.054013
2,4,37,63.971367,11.288850,4.698689,4.055223,18.586061,18.242400,57.515636,61.197197,13.694597,1.883858,6.400974
3,4,36,62.499893,12.081700,4.425854,4.508512,17.868702,18.509598,56.733430,59.383293,-1.867606,-3.462506,4.670726
4,6,51,88.805970,16.424040,6.568262,5.840533,25.406647,25.530130,80.040184,84.478540,11.079476,-0.483676,5.545160


In [118]:
# Scaling GAN data, using the same scaler as above

for key in exercise_dataframes_dict_gan.keys():
    arr = scaler.fit_transform(exercise_dataframes_dict_gan[key])
    exercise_dataframes_dict_gan[key] = pd.DataFrame(arr, columns=exercise_dataframes_dict_gan[key].columns)

exercise_dataframes_dict_gan['an_2008_exercise_synthetic'].head()

,group,no_of_sessions,age,bmi,pain_before,pain_after,func_before,func_after,QoL_before,QoL_after,pain_improvement,func_improvement,QoL_improvement
0,-0.837218,-0.699238,-0.653537,-0.614769,-0.615678,-0.440387,-0.668055,-0.614452,-0.636256,-0.650029,-0.494503,-0.668263,-0.514950
1,-0.055815,-0.803379,-0.827452,-0.908019,-0.799528,-0.826829,-0.748085,-0.833934,-0.819659,-0.786565,-0.068976,0.717571,0.612837
2,-1.618622,-1.428230,-1.440278,-1.668826,-1.295610,-1.763494,-1.364277,-1.521172,-1.461268,-1.410276,0.997637,1.531138,0.986036
3,-1.618622,-1.532372,-1.528339,-1.411734,-1.504058,-1.354138,-1.511049,-1.465949,-1.512867,-1.522115,-0.864187,-0.962794,-0.875057
4,-0.055815,0.029755,0.045961,-0.003674,0.132757,-0.151215,0.031218,-0.014976,0.024560,0.025180,0.684770,0.426749,0.065503


## SVM
* **Why choose this:** Simply because I'm already familiar with it and it is simple to implement. Also it should work OK with the data.
* **Note:** Perhaps should do hyperparameter tuning but that is computationally expensive so not implemented now. Might do that if we get better data. Right now, the data is not good so hyperparameter tuning won't make the results any better so it is just a waste of time.

### Assumptions
* trying to predict the improvement percentage based on the patients age and bmi
    * DM needs to provide that info for the problem implementation

In [119]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import r2_score
# df is the (exercise group) dataframe being used
# cols is the columns we are the predictors (age, bmi and baseline values for pain/funct/QoL)
# y is the "target" we are trying to "predict" (should be either pain_improvement, func_improvement or QoL_improvement)
def fit_svm(df, cols, y):
    # This here is our data
    X = df[cols]
    y = df[y]

    # Splitting it to training and testing
    X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.3, random_state=42)

    svr = svm.SVR()
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    r2 = r2_score(y_test, y_pred)

    return r2, svr



## SVM: Crappy data

In [120]:
results_svm_dict = {}  # Storing the results and fitted surrogates here

# These are the labels, or values we are trying to precit
# This is not the cleanest way of doing it, but it is a quick fix.
options = [['pain_before' ,'pain_improvement'], ['func_before', 'func_improvement'], ['QoL_before', 'QoL_improvement']]


for key in exercise_dataframes_dict.keys():
    temp = {}
    # Here running through the different options and fitting the surrogate for each option and collecting the info to a dictionary
    for option in options:
        features = ['age', 'bmi']
        features.append(option[0])
        r2, svr = fit_svm(exercise_dataframes_dict[key], features, option[1])
        temp[option[1]] = {'r2': r2, 'svr': svr}
    results_svm_dict[key] = temp

The structure of the dictionary is a bit confusing but should work.

In [121]:
results_svm_dict['data_an_2008_exercise']

{'pain_improvement': {'r2': -0.19639514261709845, 'svr': SVR()},
 'func_improvement': {'r2': 0.26952663538550314, 'svr': SVR()},
 'QoL_improvement': {'r2': -0.6485899251149752, 'svr': SVR()}}

## SVM: GAN-generated

NOTE: This is "predicting" after values, not the percentage  (in theory not that different, I think, since there is no new information)

Should maybe fix later

In [122]:
results_svm_dict_gan = {}  # Storing the results and fitted surrogates here

# These are the labels, or values we are trying to precit
# This is not the cleanest way of doing it, but it is a quick fix.
options = [['pain_before' ,'pain_after'], ['func_before', 'func_after'], ['QoL_before', 'QoL_after']]

for key in exercise_dataframes_dict_gan.keys():
    temp = {}
    # Here running through the different options and fitting the surrogate for each option and collecting the info to a dictionary
    for option in options:
        features = ['age', 'bmi']
        features.append(option[0])
        r2, svr = fit_svm(exercise_dataframes_dict_gan[key], features, option[1])
        temp[option[1]] = {'r2': r2, 'svr': svr}
    results_svm_dict_gan[key] = temp

In [123]:
results_svm_dict_gan['an_2008_exercise_synthetic']

{'pain_after': {'r2': 0.9443707080522713, 'svr': SVR()},
 'func_after': {'r2': 0.9011693280012288, 'svr': SVR()},
 'QoL_after': {'r2': 0.9175370303939123, 'svr': SVR()}}

## Random forests

* *Why choose it:* It should work OK with the smaller datasets, overfitting shouldn't be an issue

### TODO:
- Figure out the right number of estimators, now set to 10. Might be too simple  (Hyperparameter tuning?)

In [124]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

def fit_random_forest(df, cols, y):
    # This here is our data
    X = df[cols]
    y = df[y]

    # Splitting it to training and testing
    X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.3, random_state=42)

    rf_reg = RandomForestRegressor(n_estimators=10, random_state=42)
    rf_reg.fit(X_train, y_train)
    y_pred = rf_reg.predict(X_test)

    r2 = r2_score(y_test, y_pred)

    return r2, rf_reg

## RF: Crappy data

In [125]:
results_forest_reg_dict = {}

options = [['pain_before' ,'pain_improvement'], ['func_before', 'func_improvement'], ['QoL_before', 'QoL_improvement']]

for key in exercise_dataframes_dict.keys():
    temp = {}
    # Here running through the different options and fitting the surrogate for each option and collecting the info to a dictionary
    for option in options:
        features = ['age', 'bmi']
        features.append(option[0])
        r2, forest_reg = fit_random_forest(exercise_dataframes_dict[key], features, option[1])
        temp[option[1]] = {'r2': r2, 'forest_reg': forest_reg}
    results_forest_reg_dict[key] = temp

In [126]:
# Get the pain_improvement predictor/surrogate function or whatever it is called
results_forest_reg_dict['data_cheung_2014_exercise']['pain_improvement']['forest_reg']

RandomForestRegressor(n_estimators=10, random_state=42)

## RF: GAN-data

In [127]:
results_forest_reg_dict_gan = {}

options = [['pain_before' ,'pain_after'], ['func_before', 'func_after'], ['QoL_before', 'QoL_after']]

for key in exercise_dataframes_dict_gan.keys():
    temp = {}
    # Here running through the different options and fitting the surrogate for each option and collecting the info to a dictionary
    for option in options:
        features = ['age', 'bmi']
        features.append(option[0])
        r2, forest_reg = fit_random_forest(exercise_dataframes_dict_gan[key], features, option[1])
        temp[option[1]] = {'r2': r2, 'forest_reg': forest_reg}
    results_forest_reg_dict_gan[key] = temp

In [128]:
# Get the pain_improvement predictor/surrogate function or whatever it is called
results_forest_reg_dict_gan['cheung_2014_exercise_synthetic']['pain_after']['forest_reg']

RandomForestRegressor(n_estimators=10, random_state=42)

In [129]:
# %pip install -U prettytable

## Gradient Boosting Regression

This model is effective for handling regression tasks, especially when dealing with structured tabular data, and normally it provides better performance compared to Random Forests.

In [130]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

def fit_gradient_boosting(df, cols, y):
    X = df[cols]
    y = df[y]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    gb_reg = GradientBoostingRegressor(n_estimators=100, random_state=42)
    gb_reg.fit(X_train, y_train)

    y_pred = gb_reg.predict(X_test)

    r2 = r2_score(y_test, y_pred)

    return r2, gb_reg

### GB Regression: Crappy data

In [131]:
results_gb_dict = {}
options = [['pain_before', 'pain_improvement'], ['func_before', 'func_improvement'], ['QoL_before', 'QoL_improvement']]

for key in exercise_dataframes_dict.keys():
    temp = {}
    for option in options:
        features = ['age', 'bmi']
        features.append(option[0])
        r2, gb_reg = fit_gradient_boosting(exercise_dataframes_dict[key], features, option[1])
        temp[option[1]] = {'r2': r2, 'gb_reg': gb_reg}
    results_gb_dict[key] = temp

print(results_gb_dict['data_an_2008_exercise'])

{'pain_improvement': {'r2': -0.5459289830404972, 'gb_reg': GradientBoostingRegressor(random_state=42)}, 'func_improvement': {'r2': -0.3003585108400588, 'gb_reg': GradientBoostingRegressor(random_state=42)}, 'QoL_improvement': {'r2': -0.924609024740149, 'gb_reg': GradientBoostingRegressor(random_state=42)}}


### GB Regression: GAN Data

In [132]:
results_gb_dict_gan = {}
options = [['pain_before' ,'pain_after'], ['func_before', 'func_after'], ['QoL_before', 'QoL_after']]
for key in exercise_dataframes_dict_gan.keys():
    temp = {}
    for option in options:
        features = ['age', 'bmi']
        features.append(option[0])
        r2, gb_reg = fit_gradient_boosting(exercise_dataframes_dict_gan[key], features, option[1])
        temp[option[1]] = {'r2': r2, 'gb_reg': gb_reg}
    results_gb_dict_gan[key] = temp

print(results_gb_dict_gan['an_2008_exercise_synthetic'])

{'pain_after': {'r2': 0.9129334172860618, 'gb_reg': GradientBoostingRegressor(random_state=42)}, 'func_after': {'r2': 0.9607243685084914, 'gb_reg': GradientBoostingRegressor(random_state=42)}, 'QoL_after': {'r2': 0.9670235233679972, 'gb_reg': GradientBoostingRegressor(random_state=42)}}


# Tables

Here printing out the results to be easily viewed.

## Tables: Crap data

In [133]:
# for rounding the r2 scores to 4 decimal places
dec_places = 4

therapies = list(results_svm_dict.keys())
therapies = [t[5:] for t in therapies] # dropping "data_"

# Extracting the r2 scores for the three different objectives
svm_pain_scores = [round(results_svm_dict[key]['pain_improvement']['r2'], dec_places) for key in results_svm_dict]
svm_func_scores = [round(results_svm_dict[key]['func_improvement']['r2'], dec_places) for key in results_svm_dict]
svm_qol_scores = [round(results_svm_dict[key]['QoL_improvement']['r2'], dec_places ) for key in results_svm_dict]

forest_pain_scores = [round(results_forest_reg_dict[key]['pain_improvement']['r2'], dec_places) for key in results_forest_reg_dict]
forest_func_scores = [round(results_forest_reg_dict[key]['func_improvement']['r2'], dec_places) for key in results_forest_reg_dict]
forest_qol_scores = [round(results_forest_reg_dict[key]['QoL_improvement']['r2'], dec_places ) for key in results_forest_reg_dict]

gb_pain_scores = [round(results_gb_dict[key]['pain_improvement']['r2'], dec_places) for key in results_gb_dict]
gb_func_scores = [round(results_gb_dict[key]['func_improvement']['r2'], dec_places) for key in results_gb_dict]
gb_qol_scores = [round(results_gb_dict[key]['QoL_improvement']['r2'], dec_places) for key in results_gb_dict]

In [134]:
from prettytable import PrettyTable


# Forming the table with the info extracted above
table = PrettyTable()
table.add_column('Exercise therapy', therapies)

# Group scores for pain together
table.add_column('R2 score SVM pain', svm_pain_scores)
table.add_column('R2 score Forest pain', forest_pain_scores)
table.add_column('R2 score GB pain', gb_pain_scores)

table.add_column('R2 score SVM func', svm_func_scores)
table.add_column('R2 score Forest func', forest_func_scores)
table.add_column('R2 score GB func', gb_func_scores)

table.add_column('R2 score SVM QoL', svm_qol_scores)
table.add_column('R2 score Forest QoL', forest_qol_scores)
table.add_column('R2 score GB QoL', gb_qol_scores)

## Tables: GAN-generated

In [135]:
# for rounding the r2 scores to 4 decimal places
dec_places = 4

therapies_gan = list(results_svm_dict_gan.keys())

# Extracting the r2 scores for the three different objectives
svm_pain_scores_gan = [round(results_svm_dict_gan[key]['pain_after']['r2'], dec_places) for key in results_svm_dict_gan]
svm_func_scores_gan = [round(results_svm_dict_gan[key]['func_after']['r2'], dec_places) for key in results_svm_dict_gan]
svm_qol_scores_gan = [round(results_svm_dict_gan[key]['QoL_after']['r2'], dec_places ) for key in results_svm_dict_gan]

forest_pain_scores_gan = [round(results_forest_reg_dict_gan[key]['pain_after']['r2'], dec_places) for key in results_forest_reg_dict_gan]
forest_func_scores_gan = [round(results_forest_reg_dict_gan[key]['func_after']['r2'], dec_places) for key in results_forest_reg_dict_gan]
forest_qol_scores_gan = [round(results_forest_reg_dict_gan[key]['QoL_after']['r2'], dec_places ) for key in results_forest_reg_dict_gan]

gb_pain_scores_gan = [round(results_gb_dict_gan[key]['pain_after']['r2'], dec_places) for key in results_gb_dict_gan]
gb_func_scores_gan = [round(results_gb_dict_gan[key]['func_after']['r2'], dec_places) for key in results_gb_dict_gan]
gb_qol_scores_gan = [round(results_gb_dict_gan[key]['QoL_after']['r2'], dec_places) for key in results_gb_dict_gan]

In [136]:
from prettytable import PrettyTable

# Forming the table with the info extracted above
table_gan = PrettyTable()
table_gan.add_column('Exercise therapy', therapies_gan)

# Let's group scores for pain together, easier to compare
table_gan.add_column('R2 score SVM pain', svm_pain_scores_gan)
table_gan.add_column('R2 score Forest pain', forest_pain_scores_gan)
table_gan.add_column('R2 score GB pain', gb_pain_scores_gan)

table_gan.add_column('R2 score SVM func', svm_func_scores_gan)
table_gan.add_column('R2 score Forest func', forest_func_scores_gan)
table_gan.add_column('R2 score GB func', gb_func_scores_gan)

table_gan.add_column('R2 score SVM QoL', svm_qol_scores_gan)
table_gan.add_column('R2 score Forest QoL', forest_qol_scores_gan)
table_gan.add_column('R2 score GB QoL', gb_qol_scores_gan)

## Displaying the results for the surrogates


## Crap data

Below displaying the R2 scores for the different surrogates (copied from the code prinout below and turned into markdown with chatGPT)

| Exercise therapy        | R2 score SVM pain | R2 score Forest pain | R2 score SVM func | R2 score Forest func | R2 score SVM QoL | R2 score Forest QoL |
|-------------------------|-------------------|----------------------|-------------------|----------------------|-------------------|----------------------|
| an_2008_exercise        | -0.3256           | -0.5743              | 0.1111            | -2.9137              | -1.2644           | -2.7514              |
| cheung_2014_exercise    | -0.1702           | -0.2002              | -0.0685           | -0.2025              | -1.3582           | -1.6960              |
| evcik_2002_exercise1    | -0.3539           | -0.0076              | -5.5734           | -8.4030              | 0.0114            | 0.4109               |
| evcik_2002_exercise2    | -2.1705           | -5.7968              | -1.0676           | -1.9468              | -0.2469           | -0.7064              |
| fransen_2001_exercise1  | -1.8646           | -23.2285             | -0.6585           | -1.8131              | 0.0361            | -0.6991              |
| fransen_2001_exercise2  | -1.0527           | -4.2183              | -0.0013           | -1.7341              | -0.3203           | -0.2580              |
| jorge_2015_exercise     | -0.3791           | -1.5292              | 0.4831            | -1.3221              | 0.1860            | -0.6291              |
| lee_2008_exercise       | -1.0993           | -19.5360             | -0.0627           | -0.6748              | -0.0869           | 0.0355               |
| lee_2009_exercise       | -0.1511           | -1.4860              | -0.2022           | -0.1723              | -0.2209           | -0.0879              |
| salacinski_2012_exercise| -0.8811           | -0.3521              | -0.2532           | -0.2797              | -6.9241           | -6.8854              |
| wallis_2017_exercise    | -0.0517           | -0.4079              | 0.0555            | 0.2827               | -0.3719           | -0.3872              |


## Gan-generated (all surrogates)


|          Exercise therapy          | R2 score SVM pain | R2 score Forest pain | R2 score GB pain | R2 score SVM func | R2 score Forest func | R2 score GB func | R2 score SVM QoL | R2 score Forest QoL | R2 score GB QoL |
|------------------------------------|-------------------|----------------------|------------------|-------------------|----------------------|------------------|------------------|---------------------|-----------------|
|     an_2008_exercise_synthetic     |       **0.9444**      |        0.843         |      0.9129      |       0.9012      |        0.837         |      **0.9607**      |      0.9175      |        0.942        |      **0.967**      |
|   cheung_2014_exercise_synthetic   |       **0.4373**      |        0.3909        |     -0.0049      |       0.9205      |        **0.9412**        |      0.993       |      0.9224      |        0.9236       |      **0.9731**     |
|   evcik_2002_exercise1_synthetic   |       0.9707      |        **0.971**         |      0.9237      |       **0.9911**      |        0.9333        |      0.986       |      **0.9928**      |         0.98        |      0.9872     |
|   evcik_2002_exercise2_synthetic   |       **0.938**       |        0.9343        |       0.92       |       **0.9672**      |        0.9327        |      0.934       |      **0.9789**      |        0.9705       |      0.9457     |
|  fransen_2001_exercise1_synthetic  |       0.8766      |        0.8507        |      **0.9096**      |       0.925       |        0.9397        |      **0.9496**      |      **0.6551**      |        0.6208       |      0.3682     |
|  fransen_2001_exercise2_synthetic  |       0.4964      |        **0.5691**        |       0.43       |       0.9758      |        0.9746        |       **0.99**       |      0.9899      |        0.9945       |      **0.9947**     |
|   jorge_2015_exercise_synthetic    |       0.7986      |        **0.8563**        |      0.7469      |       **0.8913**      |        0.8689        |      0.8795      |      0.9805      |        0.973        |      **0.9831**     |
|    lee_2008_exercise_synthetic     |       0.8163      |        0.9184        |      **0.9437**      |       0.7472      |        0.8436        |      **0.857**       |      0.8743      |        **0.9459**       |      0.9398     |
|    lee_2009_exercise_synthetic     |       0.9397      |        0.9064        |      **0.9472**      |       **0.9673**      |        0.6264        |      0.7381      |      **0.912**       |        0.8836       |      0.8267     |
| salacinski_2012_exercise_synthetic |       **0.6327**      |        0.3144        |      0.3275      |       **0.0207**      |       -0.8448        |     -0.4018      |      0.9288      |        0.9506       |      **0.9681**     |
|   wallis_2017_exercise_synthetic   |       **0.832**       |        0.7696        |      0.7762      |       0.9279      |        0.9648        |      **0.9848**      |      0.9181      |        0.9662       |      **0.9827**     |


In [141]:
#print(table)
print(table_gan)

+------------------------------------+-------------------+----------------------+------------------+-------------------+----------------------+------------------+------------------+---------------------+-----------------+
|          Exercise therapy          | R2 score SVM pain | R2 score Forest pain | R2 score GB pain | R2 score SVM func | R2 score Forest func | R2 score GB func | R2 score SVM QoL | R2 score Forest QoL | R2 score GB QoL |
+------------------------------------+-------------------+----------------------+------------------+-------------------+----------------------+------------------+------------------+---------------------+-----------------+
|     an_2008_exercise_synthetic     |       0.9444      |        0.843         |      0.9129      |       0.9012      |        0.837         |      0.9607      |      0.9175      |        0.942        |      0.967      |
|   cheung_2014_exercise_synthetic   |       0.4373      |        0.3909        |     -0.0049      |       0.920

# Picking the best performing surrogates

Here yada yada about which surrogate performed best for each dataset

# Problem formulation

### Preferences provided by the DM in the original paper
Acceptable minimum and maximum ranges for the objectives (**Can't use it exactly like this, need to modify**)

| |Costs (€) | Pain change (%) | Function change (%) | Supervised sessions | Period (weeks) |
| -------|------------|------------------|----------------------|----------------------|----------------|
| Iteration 1 | 300 | +30% | +25% | 0 | 8 |
| | 600 | +15% | +15% | 15 | 26 |
| Iteration 2  | 200 | +25% | +40% | 0 | 12 |
| | 500 | +15% | +15% | 30 | 26 |


Suggestion by P as DM (Didn't change much):

| |Costs (€) | Pain change (%) | Function change (%) | Supervised sessions | Period (weeks) |
| -------|------------|------------------|----------------------|----------------------|----------------|
| Iteration 1 | 200 | +30% | +25% | 0 | 8 |
| | 800 | +15% | +15% | 15 | 24 |
| Iteration 2  | 150 | +25% | +40% | 0 | 12 |
| | 600 | +15% | +15% | 20 | 24 |


Shall we add QoL? Just leave it?

And I am bit confuse about how to solve this now. Can we use orginal code from the paper? or are we going to do seperate implementation using DESDEO?

In [138]:
# %pip install desdeo

In [139]:
#I did not run the code and it seems colab has an issue of loading synthetic data

#Just Seperate Implementation using DESDEO
from desdeo.problem import ScalarDataProblem, ScalarMOProblem
from desdeo.variables import variable_builder
from desdeo.metrics import Hypervolume

ranges_iteration_1 = [
    (200, 800),    # Costs (€)
    (30, 15),      # Pain change (%)
    (25, 15),      # Function change (%)
    (0, 15),       # Supervised sessions
    (8, 24)        # Period (weeks)
]

ranges_iteration_2 = [
    (150, 600),    # Costs (€)
    (25, 15),      # Pain change (%)
    (40, 15),      # Function change (%)
    (0, 20),       # Supervised sessions
    (12, 24)       # Period (weeks)
]

# Bit changed from orginal paper
variable_type = ["int", "float", "float", "int", "int"]
variable_ranges = [(150, 800), (-30, 30), (-30, 30), (0, 20), (8, 24)]

problem_iteration_1 = ScalarDataProblem(variable_builder(variable_type, variable_ranges),
                                        ["Costs (€)", "Pain change (%)", "Function change (%)",
                                         "Supervised sessions", "Period (weeks)"],
                                        ["minimize", "minimize", "minimize", "minimize", "minimize"],
                                        variables_description=["Costs associated with the therapy",
                                                               "Percentage change in pain",
                                                               "Percentage change in function",
                                                               "Number of supervised sessions",
                                                               "Duration of therapy in weeks"],
                                        ideal=[200, 30, 25, 0, 8],
                                        nadir=[800, 15, 15, 15, 24],
                                        constraints=ranges_iteration_1)

problem_iteration_2 = ScalarDataProblem(variable_builder(variable_type, variable_ranges),
                                        ["Costs (€)", "Pain change (%)", "Function change (%)",
                                         "Supervised sessions", "Period (weeks)"],
                                        ["minimize", "minimize", "minimize", "minimize", "minimize"],
                                        variables_description=["Costs associated with the therapy",
                                                               "Percentage change in pain",
                                                               "Percentage change in function",
                                                               "Number of supervised sessions",
                                                               "Duration of therapy in weeks"],
                                        ideal=[150, 25, 40, 0, 12],
                                        nadir=[600, 15, 15, 20, 24],
                                        constraints=ranges_iteration_2)


problems = [problem_iteration_1, problem_iteration_2]
scalar_mo_problem = ScalarMOProblem(problems)

hypervolume_metric = Hypervolume(scalar_mo_problem)

# Can change algorithm. Just used NSGA2
from desdeo.optimizers.NSGA2 import NSGA2

optimizer = NSGA2(scalar_mo_problem)

n_iterations = 100
approximation = optimizer.minimize(n_iterations)

for i, problem in enumerate(problems):
    print(f"Iteration {i+1}:")
    print(f"  Number of solutions: {len(approximation.objectives[i])}")
    print(f"  Hypervolume: {hypervolume_metric(approximation.objectives[i])}")

ModuleNotFoundError: No module named 'desdeo.problem'